In [1]:
#Importing all the packages for the execution of the code
import pandas as pd
import numpy as np
import collections
import itertools


In [2]:
#importing the csv file and converting it to dataframe
dataframe_balance= pd.read_csv("balance-scale.csv")

dict_testset_class={}

#Renaming the column values R to Positive and l to Negative
dataframe_balance=dataframe_balance.replace('R','Positive')
dataframe_balance=dataframe_balance.replace('L','Negative')

#splitting into training and testing dataframe. 
df_training=dataframe_balance.sample(frac=0.7)

df_testing=pd.concat([dataframe_balance, df_training, df_training]).drop_duplicates(keep=False)
dict_testset_class=df_testing['Class'].to_dict()


# calculating the unique values of each column
dictionary_unique_values=df_training.T.apply(lambda x: x.nunique(), axis=1).to_dict()





In [3]:
#calculating the prior probability of the class label values

#Storing unique values of the class label.
list_unique_values=[]
for i in df_training['Class']:
    list_unique_values.append(i)
unique=set(list_unique_values) 

dict_prior={}

#Converting the prior probability dataframe to dictionary
prior = df_training.groupby("Class").size().div(len(df_training))
dict_prior=prior.to_dict()

#calculating the occurence of each value of class label.
count_class_values={}
count_class_values = df_training.groupby("Class").size().to_dict()


In [4]:
#Calculating the conditional probabilities

colum_value_count={}
for column in df_training.columns:
    if column != "Class":
        sd=(df_training.groupby(column)["Class"].value_counts()+1).to_dict() #adding 1 to the count i.e. adding mp to the count. where m is the number of unique values of the feature and p is inverse of m.  
        colum_value_count[column]=sd

print(colum_value_count)
    



{'LWght': {(1, 'Positive'): 71, (1, 'Negative'): 10, (2, 'Positive'): 32, (2, 'Negative'): 11}, 'LDist': {(1, 'Positive'): 33, (2, 'Positive'): 29, (2, 'Negative'): 6, (3, 'Positive'): 18, (3, 'Negative'): 8, (4, 'Positive'): 15, (4, 'Negative'): 4, (5, 'Positive'): 11, (5, 'Negative'): 5}, 'RWght': {(1, 'Negative'): 12, (1, 'Positive'): 12, (2, 'Positive'): 18, (2, 'Negative'): 5, (3, 'Positive'): 25, (3, 'Negative'): 4, (4, 'Positive'): 28, (4, 'Negative'): 2, (5, 'Positive'): 23}, 'RDist': {(1, 'Positive'): 13, (1, 'Negative'): 10, (2, 'Positive'): 19, (2, 'Negative'): 7, (3, 'Positive'): 21, (3, 'Negative'): 4, (4, 'Positive'): 26, (4, 'Negative'): 2, (5, 'Positive'): 27}}


In [5]:
#Calculating the m-estimate conditional probability by dividing the count of feature value given the class label, by the sum of the count of the class
#label occuring in the training dataset and the unique values of the current feature.
dictionary_conditional_prob={}
dictionary_conditional_prob_col={}
for key,value in colum_value_count.items():
    for key1,value1 in value.items():
        ans=(value1/(count_class_values[key1[1]]+dictionary_unique_values[key]))
        dictionary_conditional_prob[key1]=ans
    dictionary_conditional_prob_col.setdefault(key,{}).update(dictionary_conditional_prob)
print(dictionary_conditional_prob_col)    
 
        
        

{'LWght': {(1, 'Positive'): 0.6893203883495146, (1, 'Negative'): 0.47619047619047616, (2, 'Positive'): 0.3106796116504854, (2, 'Negative'): 0.5238095238095238}, 'LDist': {(1, 'Positive'): 0.3113207547169811, (1, 'Negative'): 0.47619047619047616, (2, 'Positive'): 0.27358490566037735, (2, 'Negative'): 0.25, (3, 'Positive'): 0.16981132075471697, (3, 'Negative'): 0.3333333333333333, (4, 'Positive'): 0.14150943396226415, (4, 'Negative'): 0.16666666666666666, (5, 'Positive'): 0.10377358490566038, (5, 'Negative'): 0.20833333333333334}, 'RWght': {(1, 'Positive'): 0.11320754716981132, (1, 'Negative'): 0.5, (2, 'Positive'): 0.16981132075471697, (2, 'Negative'): 0.20833333333333334, (3, 'Positive'): 0.2358490566037736, (3, 'Negative'): 0.16666666666666666, (4, 'Positive'): 0.2641509433962264, (4, 'Negative'): 0.08333333333333333, (5, 'Positive'): 0.2169811320754717, (5, 'Negative'): 0.20833333333333334}, 'RDist': {(1, 'Positive'): 0.12264150943396226, (1, 'Negative'): 0.4166666666666667, (2, 'Pos

In [6]:
# calculating the probability of the testing dataset instances and alloting the class  with maximum probability.

#print(df_testing)
dictionary_column_index={}
dict_predicted_label={}
prob_R={}
prob_L={}

#list of columns other than Class Column
column_list=["LWght","LDist","RWght","RDist"]
for m in column_list:
    col_dex=[df_testing.columns.get_loc(m)]

#Creating the dictionary to store index of the columns and colymn name un the form of key value pair
for m in column_list:
    dictionary_column_index[df_testing.columns.get_loc(m)]=m


#Iterating row by column and multiplying probability of each feature value in a particular row and then multiplying the row
#probability with the prior of the class labels to find the maximum probability and then classify.
for index,row in df_testing.iterrows():
    cal_R=1
    cal_L=1
    for colum in range(1,5):
        
        cal_R=cal_R*dictionary_conditional_prob_col[dictionary_column_index[colum]][(row[dictionary_column_index[colum]],'Positive')]
        cal_L=cal_L*dictionary_conditional_prob_col[dictionary_column_index[colum]][(row[dictionary_column_index[colum]],'Negative')]   
    
    prob_R=count_class_values['Positive']*cal_R
    prob_L=count_class_values['Negative']*cal_L
    
    label='Positive' if prob_R>prob_L else 'Negative'
    dict_predicted_label[index]=label
print(dict_predicted_label)        



{1: 'Positive', 5: 'Positive', 6: 'Positive', 11: 'Positive', 13: 'Positive', 17: 'Positive', 21: 'Positive', 22: 'Positive', 27: 'Positive', 29: 'Positive', 30: 'Positive', 31: 'Positive', 41: 'Positive', 43: 'Positive', 47: 'Negative', 51: 'Negative', 53: 'Positive', 54: 'Positive', 57: 'Positive', 63: 'Positive', 66: 'Positive', 68: 'Positive', 70: 'Negative', 71: 'Negative', 79: 'Positive', 87: 'Positive', 88: 'Positive', 94: 'Positive', 95: 'Positive', 96: 'Negative', 98: 'Positive', 101: 'Positive', 103: 'Positive', 110: 'Positive', 111: 'Positive', 114: 'Positive', 115: 'Negative', 119: 'Positive', 120: 'Positive', 126: 'Positive', 128: 'Positive', 134: 'Positive', 136: 'Positive', 138: 'Negative', 141: 'Negative', 145: 'Positive', 148: 'Positive', 150: 'Positive', 158: 'Positive', 164: 'Negative', 165: 'Negative', 170: 'Positive'}


In [7]:
#Converting predicted dictionary to the dataframe and creatig a column named Predicted Label
df_predicted=pd.DataFrame.from_dict(dict_predicted_label,orient='index')
df_predicted.columns=["Predicted Label"]



In [11]:
#concatinating df_testing with predicted dataframe to create a Resultant csv. 
df_bb=df_testing.join(df_predicted)
df_bb.to_csv('Predictions.csv', sep='\t', encoding='utf-8')

df_bb


,Class,LWght,LDist,RWght,RDist,Predicted Label
1,Positive,1,1,1,3,Positive
5,Positive,1,1,2,2,Positive
6,Positive,1,1,2,3,Positive
11,Positive,1,1,3,3,Positive
13,Positive,1,1,3,5,Positive
17,Positive,1,1,4,4,Positive
21,Positive,1,1,5,3,Positive
22,Positive,1,1,5,4,Positive
27,Positive,1,2,1,5,Positive
29,Positive,1,2,2,3,Positive


In [9]:
#Calculating the Accuracy,Specificityc and Sensitvity by calculating the number of true positive, true negative, false positve and
#false negative found.
TPC=0
TNC=0
FPC=0
FNC=0

for key in dict_testset_class.keys():
    for key1 in dict_predicted_label.keys():
        if key == key1:
            if dict_testset_class[key] == 'Positive' and dict_predicted_label[key] == dict_testset_class[key]:
                TPC+=1
            elif dict_testset_class[key] == 'Negative' and dict_predicted_label[key] == dict_testset_class[key]:
                TNC+=1
            elif dict_testset_class[key] == 'Positive' and dict_predicted_label[key] != dict_testset_class[key]:
                FPC+=1
            elif dict_testset_class[key] == 'Negative' and dict_predicted_label[key] != dict_testset_class[key]:    
                FNC+=1
print(TPC,TNC,FPC,FNC)                
                
                

39 9 1 3


In [10]:
Sensitivity=TPC/(TPC+FNC)
Specificity=TNC/(TNC+FPC)
print(Sensitivity,"Sensitivity")
print(Specificity,"Specificity")
Accuracy=(TPC+TNC)/(TPC+TNC+FPC+FNC)
print(Accuracy,"Accuracy")

0.9285714285714286 Sensitivity
0.9 Specificity
0.9230769230769231 Accuracy
